In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 請輸入資料夾之所在位置
FOLDERNAME = 'Colab\ Notebooks/SC201_L15'
assert FOLDERNAME is not None, "[!] Enter the foldername."

In [ ]:
%cd drive/MyDrive/$FOLDERNAME
BATCH_SIZE = 64

In [ ]:
# Calculate the mean/std for Training Dataset
# You will see the mean/std of your data at the end of this cell
"""
import torchvision.datasets as dset
from torch.utils.data import DataLoader
import torchvision.transforms as T

transform = T.Compose([T.Resize((64, 64)), T.ToTensor()])
training_set = dset.ImageFolder('train/', transform=transform)
train_loader = DataLoader(training_set, batch_size=len(training_set))
images, labels = next(iter(train_loader))
mean = images.mean()
std = images.std()
"""

In [ ]:
import torch
mean = torch.tensor(0.4492)
std = torch.tensor(0.2480)
# Data Pre-processing by PyTorch
# import ...

In [ ]:
%ls
# Load Your Data by PyTorch
# import ...

In [ ]:
# Create Mini-batches by PyTorch
mini_trains = None
mini_vals = None
# import ...

In [ ]:
# See if GPU is available
# To turn GPU on: Edit -> Notebook Setting -> GPU

import torch

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

In [ ]:
"""
Plot some training data! 
We need to denormalize PyTorch Tensor first, 
and then change it to images with RGB channel
"""

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
classes = ['cat','dog']

# Reverse the normalized value to 0-255
def denormalize(image):
  image = T.Normalize(-mean/std,1/std)(image) #denormalize
  image = image.permute(1,2,0) #Changing from 3x224x224 to 224x224x3
  image = torch.clamp(image,0,1)
  return image

# Display an image
def imshow(img):
    img = denormalize(img) 
    plt.imshow(img)

dataiter = iter(mini_trains)
images, labels = dataiter.next()
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 8))
# display 20 images
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title("{} ".format( classes[labels[idx]]))

In [ ]:
import torch.nn as nn

In [ ]:
model = nn.Sequential(
    # Your Model Here
    ##################
    #                #
    #      TODO:     #
    #                #      
    ##################
    pass
)

In [ ]:
from torchsummary import summary
summary(model.cuda(), (3, 64, 64))

In [ ]:
import torch.optim as optim
learning_rate = 1e-4

In [ ]:
# Choose your optimizer
##################
#                #
#      TODO:     #
#                #      
##################

In [ ]:
"""
Training Process begins!
We will be using your model and optimizer to 
call forward pass / backward pass.
However, which loss function should you choose?
Let's try out! You will see the difference between different loss function in decreasing training cost
"""

In [ ]:
def train(model, optimizer, epochs=1):
  for epoch in range(epochs):
    print('----------------------------')
    print('EPOCH: {}'.format(epoch+1))
    for t, (x, y) in enumerate(mini_trains):  
      # Turn on the training mode on your model
      model.train()
      # move tensors to GPU if CUDA is available
      x = x.to(device)
      y = y.to(device)

      # forward pass: compute predicted outputs by passing inputs to the model
      scores = model(x)
      
      # calculate the batch loss
      # Choose your loss function!
      loss_function = nn.CrossEntropyLoss()
      loss = loss_function(scores, y)

      # clear the gradients of all optimized variables
      optimizer.zero_grad()

      loss.backward()

      # perform a single optimization step (parameter update)
      optimizer.step()

      if t % 7 == 0:
        print('Iteration %d, loss = %.4f' % (t, loss.item()))
        evaluate_predictor(mini_vals, model)

In [ ]:
def evaluate_predictor(loader, model):
  # Turn on evaluation mode on your model
  model.eval()
  val_loss = 0
  val_acc = 0
  with torch.no_grad():
    for x, y in loader:
      # move tensors to GPU if CUDA is available
      x = x.to(device)
      y = y.to(device)
      # forward pass: compute predicted outputs by passing inputs to the model
      scores = model(x)
      
      # Choose your loss function!
      loss_function = nn.CrossEntropyLoss()
      loss = loss_function(scores, y)

      val_loss += loss.item()*x.size(0)
      prediction = scores.data.max(1)[1]
      acc = prediction.eq(y).sum().item()
      val_acc += acc

    # calculate average losses
    val_loss = val_loss / len(mini_vals.dataset)
    val_acc = val_acc / len(mini_vals.dataset)
    print(f'Loss: {val_loss} / Acc: {val_acc}')

In [ ]:
train(model, optimizer, epochs=6)